In [1]:
# Dependencies
from config import homeDepot_API
from serpapi import GoogleSearch
import pandas as pd
import numpy as np
from datetime import datetime
from openpyxl import load_workbook
import os
os.getcwd()


'C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\Resources'

STEP 1: EXTRACT

In [2]:
# Creating 'material_description' and 'product_id' by extracting from Home Depot. The 'product_id is what is used to query.'

material_name = []
product_id = []

ids = ['100400409', '100400406', '100137321', '100400405', 
       '202241090', '100144234', '202077145', '202077147', 
       '202288508', '203401613', '202304641', '202304645', 
       '202019377', '202316275', '202019375', '202316276']

for i in ids:

    params = {
      "engine": "home_depot_product",
      "product_id": i,
      "api_key": homeDepot_API
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    products = results['product_results']
    
    material_name.append(products['title'])
    product_id.append(products['product_id'])    

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


In [3]:
# This function uses the material names extracted from Home Depot as query (Q) and returns results (R)
def searcher(Q, R):
    params = {
      "engine": "google",
      "q": Q,
      "api_key": homeDepot_API
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    products = results["organic_results"]
    
# It will grab up to 10 prices per result but some suppliers don't list their prices online so it 
# appends 'NaN' in place of missing prices   
    for p in products[0:10]:
        try:

            title = p['title']
            link = p['link']
            price = (p['rich_snippet']['top']['detected_extensions']['price'])            
    
            result = {'Manufactures Description': title, 'Link': link, 'Material Price': price, 'VENDOR': link}
            R.append(result)
                        
        except (KeyError):
            price = 'NaN'
            result = {'Manufactures Description': title, 'Link': link, 'Material Price': price, 'VENDOR': link}
            R.append(result)

STEP 2: TRANSFORM

In [4]:
# Extracts raw data to create remaining tables

R = []
for i in range(16):
    R_i = []
   
    searcher(material_name[i], R_i)
    R.append(R_i)

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


In [5]:
# Creates cleaned data frames from raw data
all_prices = []
lowest_prices = []

for i in range(len(R)):
    df = pd.DataFrame(R[i]).assign(material_description=material_name[i])
    df['product_id'] = product_id[i]
# Create a 'Vendor location' column    
    df['vendor'] = (np.where(df['VENDOR'].str.contains('.'), df['VENDOR'].str.split('.').str[1], df['VENDOR']))
    df.drop(columns =['VENDOR'], inplace = True)
# Change data type of price to numeric and drop NaN's    
    df['Material Price'] = df['Material Price'].apply(pd.to_numeric, args=('coerce',))
    df.dropna(inplace=True)
    
# Filter out scaling issues
    if i == 0:
        df.drop(df[df['Material Price'] < 2.60].index, inplace = True)
        df.drop(df[df['Material Price'] > 75].index, inplace = True)
    if i == 1 or i ==3:
        df.drop(df[df['Material Price'] < 1.30].index, inplace = True)
        df.drop(df[df['Material Price'] > 75].index, inplace = True)  
    if i == 4 or i == 7:
        df.drop(df[df['Material Price'] > 5].index, inplace = True) 
    if i == 9 or i == 10:
        df.drop(df[df['Material Price'] < 50].index, inplace = True)
    if i == 11 or i == 12:
        df.drop(df[df['Material Price'] < 75].index, inplace = True)   
    if i == 13 or i == 14 or i == 15:
        df.drop(df[df['Material Price'] < 130].index, inplace = True)  
           
    df["Formatted Datetime"] = datetime.now().strftime("%m/%d/%Y %H:%M:%S")
    df['Materials Description'] = df['material_description']
    df['Product id'] = df['product_id']
    df.drop(columns =['Manufactures Description', 'material_description', 'product_id'], inplace = True)
    df = df.iloc[:,[4,5,1,2,0,3]]
    
    lowest_prices.append(df.min())
    all_prices.append(df)
clean_df = pd.concat(all_prices, axis=0, ignore_index=True)
working_df = pd.DataFrame(lowest_prices)
working_df = working_df.iloc[[3,1,0,5,8,4,2,6,7,12,13,14,15,9,10,11],:]
working_df = working_df.reset_index(drop=True)

In [6]:
headquarter = clean_df.iloc[:,[3]]
locations = headquarter["vendor"]

headquarters = []

for location in locations:
    params = {
      "q": f"where is {location} headquarters",
      "location": "United States",
      "hl": "en",
      "gl": "us",
      "api_key": homeDepot_API
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    location_results = results["organic_results"]
    headquarters.append(location_results[1]['snippet_highlighted_words'])

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
h

In [7]:
clean_df = clean_df.assign(location=headquarters).iloc[:,[0,1,2,3,6,4,5]].reset_index(drop=True)
clean_df

,Materials Description,Product id,Material Price,vendor,location,Link,Formatted Datetime
0,1 in. x 10 ft. Electric Metallic Tube (EMT) Co...,100400409,20.62,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/1-in-x-10-ft-Elect...,11/03/2022 16:27:17
1,1 in. x 10 ft. Electric Metallic Tube (EMT) Co...,100400409,17.55,wilmar,[28 Biopolis Road Singapore 138568],https://www.wilmar.com/Sku/24-87610/allied-tub...,11/03/2022 16:27:17
2,3/4 in. x 10 ft. Electric Metallic Tube (EMT) ...,100400406,11.91,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/3-4-in-x-10-ft-Ele...,11/03/2022 16:27:17
3,3/4 in. x 10 ft. Electric Metallic Tube (EMT) ...,100400406,10.47,ebarnett,"[Jacksonville, Florida]",https://www.ebarnett.com/Sku/2487616/34-in-x-1...,11/03/2022 16:27:17
4,3/4 in. x 10 ft. Electric Metallic Tube (EMT) ...,100400406,8.60,gordonelectricsupply,"[Kankakee, 1290 N Hobbie Ave. HQ]",https://www.gordonelectricsupply.com/p/Allied-...,11/03/2022 16:27:17
5,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...,100137321,3.98,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/Halex-1-2-in-Elect...,11/03/2022 16:27:17
6,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...,100137321,8.54,amazon,[the National Landing neighborhood of Arlington],https://www.amazon.com/Halex-26270-STEEL-CONNE...,11/03/2022 16:27:17
7,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...,100137321,5.99,lowes,[1000 Lowe's Blvd. Mooresville NC 28117],https://www.lowes.com/pd/Sigma-Electric-ProCon...,11/03/2022 16:27:17
8,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...,100137321,2.10,wilmar,[28 Biopolis Road Singapore 138568],https://www.wilmar.com/Sku/202077151/halex-12-...,11/03/2022 16:27:17
9,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,100400405,7.22,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/1-2-in-x-10-ft-Ele...,11/03/2022 16:27:17


In [8]:

location_add = clean_df.iloc[:,[3,4,5,6]].reset_index(drop=True)
working_df = working_df.merge(location_add, how='outer')
working_df = working_df.iloc[[0,1,2,6,7,10,3,9,8,11,12,13,14,15,16,17],[0,1,2,3,6,4,5]].reset_index(drop=True) 
working_df

,Materials Description,Product id,Material Price,vendor,location,Link,Formatted Datetime
0,1/2 in. x 10 ft. Electric Metallic Tube (EMT) ...,100400405,4.86,ebarnett,"[Jacksonville, Florida]",https://www.ebarnett.com/Sku/2487615/12-in-x-1...,11/03/2022 16:27:17
1,3/4 in. x 10 ft. Electric Metallic Tube (EMT) ...,100400406,8.60,ebarnett,"[Jacksonville, Florida]",https://www.ebarnett.com/Sku/2487616/34-in-x-1...,11/03/2022 16:27:17
2,1 in. x 10 ft. Electric Metallic Tube (EMT) Co...,100400409,17.55,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/1-in-x-10-ft-Elect...,11/03/2022 16:27:17
3,1/2 in. Electrical Metallic Tube (EMT) Set-Scr...,100137321,2.10,amazon,[the National Landing neighborhood of Arlington],https://www.amazon.com/Halex-26270-STEEL-CONNE...,11/03/2022 16:27:17
4,3/4 in. Standard Fitting Electric Metallic Tub...,202288508,2.98,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/Halex-3-4-in-Stand...,11/03/2022 16:27:17
5,1 in. Electrical Metallic Tube (EMT) Set-Screw...,202077147,1.60,com/p/steel-city-emt-conduit-connector-7859911...,"[Steel City, connector, EMT conduit, Connector...",https://killingworthtruevalue.com/p/steel-city...,11/03/2022 16:27:17
6,1/2 in. Electric Metallic Tube (EMT) Set-Screw...,100144234,2.10,amazon,[the National Landing neighborhood of Arlington],https://www.amazon.com/Halex-26270-STEEL-CONNE...,11/03/2022 16:27:17
7,3/4 in. Electrical Metallic Tube (EMT) Set-Scr...,202077145,4.60,amazon,[the National Landing neighborhood of Arlington],https://www.amazon.com/Halex-Electrical-Set-Sc...,11/03/2022 16:27:17
8,1 in. Electric Metallic Tube (EMT) Set-Screw C...,202241090,1.15,homedepot,"[2455 Paces Ferry Rd SE, Atlanta, GA 30339]",https://www.homedepot.com/p/Halex-1-in-Electri...,11/03/2022 16:27:17
9,250 ft. 14/2 Romex SIMpull Solid NM-B W/G Wire,202019377,84.99,amazon,[the National Landing neighborhood of Arlington],https://www.amazon.com/SOUTHWIRE-COMPANY-28827...,11/03/2022 16:27:17


STEP 3: LOAD

In [8]:
# Send a CSV and Excell copies to my computer

xlFile = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\Estimator_Worksheet-1.xlsx"

book = load_workbook(xlFile)
writer = pd.ExcelWriter(xlFile, engine = 'openpyxl')
writer.book = book

#with pd.ExcelWriter(xlFile) as writer:
working_df.to_excel(writer, sheet_name='Working_Table6')
clean_df.to_excel(writer, sheet_name='Cleaned_Data6')
writer.save()
writer.close()
    
clean_csv = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\NewDataFrames\\clean_df6.csv"
clean_df.to_csv(clean_csv, index=False)

working_csv = "C:\\Users\\kijah\\Documents\\Data_Bootcamp\\Estimator_Project\\NewDataFrames\\working_df6.csv"
working_df.to_csv(working_csv, index=False)

In [9]:
# # This code is for trouble shooting and adjusting scale

# pd.set_option('display.max_colwidth', None)
# emt1 = pd.DataFrame(R[0]).assign(material_description=material_name[0])
# emt34 = pd.DataFrame(R[1]).assign(material_description=material_name[1])
# con12 = pd.DataFrame(R[2]).assign(material_description=material_name[2])
# emt12 = pd.DataFrame(R[3]).assign(material_description=material_name[3])
# cup1 = pd.DataFrame(R[4]).assign(material_description=material_name[4])
# cup12 = pd.DataFrame(R[5]).assign(material_description=material_name[5])
# con34 = pd.DataFrame(R[6]).assign(material_description=material_name[6])
# con1 = pd.DataFrame(R[7]).assign(material_description=material_name[7])
# cup34 = pd.DataFrame(R[8]).assign(material_description=material_name[8])
# wir14 = pd.DataFrame(R[9]).assign(material_description=material_name[9])
# wir12 = pd.DataFrame(R[10]).assign(material_description=material_name[10])
# wir10 = pd.DataFrame(R[11]).assign(material_description=material_name[11])
# rom142 = pd.DataFrame(R[12]).assign(material_description=material_name[12])
# rom143 = pd.DataFrame(R[13]).assign(material_description=material_name[13])
# rom122 = pd.DataFrame(R[14]).assign(material_description=material_name[14])
# rom123 = pd.DataFrame(R[15]).assign(material_description=material_name[15])